## Импорт биолиотек

In [1]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto

## Обновление списков

In [2]:
def get_available_categories(category, mergetable):
    if category == "Ген":
        typen = "gene"
    elif category == "Полное название":
        typen = "fullname"
    elif category == "UniProt ID":
        typen = "NCBIID"
    
    
    if category is not None:
        available_prot = mergetable[typen].unique()
    else:    available_prot = mergetable['gene'].unique()
        
        
    return [{'label': categ, 'value': categ} for categ in sorted(available_prot)]

## Функции графиков

In [3]:
def create_dis_net(prot, mergetable):
    if mergetable['gene'].eq(prot).sum() > 0:
        dfprot1 = mergetable[mergetable.gene == prot]
    elif mergetable['fullname'].eq(prot).sum() > 0:
        dfprot1 = mergetable[mergetable.fullname == prot]
    elif mergetable['NCBIID'].eq(prot).sum() > 0:
        dfprot1 = mergetable[mergetable.NCBIID == prot]
    
    listofnodes = dfprot1['gene'].unique().tolist() + dfprot1['diseases'].values.tolist()
    listofweights = dfprot1['score'].values.tolist()
    
    if len(listofnodes) <=11:
        pass
    else:
        listofnodes = listofnodes[:11]
        
    pp = [
    { 'data': {'id': listofnodes[0]},
    'classes': 'rectangle purple'}
    ]
    
    nodes = [
    {
        'data': {'id': x, 'label': x, 'weight': weight}
    }
    for x, weight in zip(listofnodes[1:], listofweights)]
    
    edges = [
    {'data': {'source': listofnodes[0], 'target': target,
             'weight': weight}}
    for target, weight in zip(listofnodes[1:], listofweights)
    ]
    elements = pp + nodes + edges
    return elements

In [4]:
def create_tis_net(prot, mergetable):
    
    if mergetable['gene'].eq(prot).sum() > 0:
        dfprot1 = mergetable[mergetable.gene == str(prot)]
    elif mergetable['fullname'].eq(prot).sum() > 0:
        dfprot1 = mergetable[mergetable.fullname == str(prot)]
    elif mergetable['NCBIID'].eq(prot).sum() > 0:
        dfprot1 = mergetable[mergetable.NCBIID == str(prot)]
               
    listofnodes = dfprot1['gene'].unique().tolist() + dfprot1['tissues'].values.tolist()

    pp = [
    { 'data': {'id': listofnodes[0]},
    'classes': 'rectangle  purple'}
    ]
    
    nodes = [
    {
        'data': {'id': x, 'label': x},
        'classes': 'triangle pink'
    }
    for x in listofnodes[1:]]
    
    edges = [
    {'data': {'source': listofnodes[0], 'target': target}}
    for target in listofnodes[1:]
    ]
    
    elements = pp + nodes + edges
    
    return elements

In [5]:
def create_prot_net(dis, mergetable, prottype):
    dfprot1 = mergetable[mergetable.diseases == str(dis)]
    if prottype == 'Ген':
        listofnodes = dfprot1['diseases'].unique().tolist() + dfprot1.gene.unique().tolist()
    if prottype == 'Полное название':
        listofnodes = dfprot1['diseases'].unique().tolist() + dfprot1.fullname.unique().tolist()
    if prottype == 'UniProt ID':
        listofnodes = dfprot1['diseases'].unique().tolist() + dfprot1.NCBIID.unique().tolist()
    
    listofweights = dfprot1['score'].values.tolist()
    
    if len(listofnodes) <=12:
        pass
    else:
        listofnodes = listofnodes[:11]
        
    pp = [
    { 'data': {'id': listofnodes[0]}, 'classes': 'purple'}
    ]
    
    nodes = [
    {
        'data': {'id': x, 'label': x, 'weight': weight},
        'classes': 'rectangle'
    }
    for x, weight in zip(listofnodes[1:], listofweights)]
    
    edges = [
    {'data': {'source': listofnodes[0], 'target': target,
             'weight': weight}}
    for target, weight in zip(listofnodes[1:], listofweights)
    ]
    
    elements = pp + nodes + edges
    
    return elements

## Приложение

In [6]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

## Загрузка данных

In [7]:
distable = pd.read_csv('human_disease_experiments_full.tsv', 
                       sep = '\t', 
                       names = ['ENSP', 'gene', 'DOID', 'diseases', 'TIGA', 'MenRankScore', 'score'])

In [8]:
distable = distable[distable['diseases'].str.contains('ICD10')==False]

In [9]:
distable = distable[distable['diseases'].eq('Disease')==False]

In [10]:
distable = distable.sort_values(by=['gene', 'score'], ascending=False)

In [11]:
prottable = pd.read_csv('gene-uniprot.txt',
                       sep = '\t',
                       names = ['gene', 'NCBIID', 'fullname', 'tissues'])

In [12]:
result = pd.merge(distable, prottable, how="right", on=['gene'])

In [13]:
mergetable = result.dropna()

In [14]:
available_prot = sorted(mergetable['gene'].unique())
available_dis = sorted(mergetable['diseases'].unique())

## Фильтры

In [15]:
protnametype = dbc.Col([
        dcc.Dropdown(
            id="protnametype_dropdown",
            value='Ген',
            options=['Ген', 'Полное название', 'UniProt ID']
        ),
    ],
    style={'width': '400px'},
)

In [16]:
protdis_filter1 = dbc.Col([
        dcc.Dropdown(
            id="protdis_dropdown1",
            value=available_prot[0],
            options=[{'label': category, 'value': category} for category in available_prot]
        ),
    ],
    style={'max-width': '100%'},
)

In [17]:
prottis_filter1 = dbc.Col([
        dcc.Dropdown(
            id="prottis_dropdown1",
            value='ALB',
            options=[{'label': category, 'value': category} for category in available_prot]
        ),
    ],
    style={'max-width': '100%'},
)

In [18]:
disprot_filter1 =dbc.Col([
        dcc.Dropdown(
            id="disprot_dropdown1",
            value='Disease of anatomical entity',
            options=[{'label': category, 'value': category} for category in available_dis]
        ),
    ],
    style={'max-width': '100%'},
)

## Карточки

In [19]:
prdiscard1 = dbc.Card(
    [
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.Label("Белок — Заболевания",
                               style={'font-size': 24,
                                      'text-align': 'left',
                                      },
                               ),
                ], width=6),
                dbc.Col([
                    protdis_filter1,
                ], width=6)
            ]),
            cyto.Cytoscape(
                id='cytoscape-prdiscard1-callbacks',
                layout={'name': 'cose'},
                stylesheet=[
                    {'selector': 'node',
                     'style': {
                         'label': 'data(label)'}
                    },
                    {
                        'selector': '[weight > 0]',
                        'style': {
                            'background-color': '#dde9f4',
                            'line-color': '#dde9f4'
                        }
                    },
                    {
                        'selector': '[weight > 0.54]',
                        'style': {
                            'background-color': '#bed6ea',
                            'line-color': '#bed6ea'
                        }
                    },
                    {
                        'selector': '[weight > 0.693]',
                        'style': {
                            'background-color': '#9fc2e0',
                            'line-color': '#9fc2e0'
                        }
                    },
                    {
                        'selector': '[weight > 0.846]',
                        'style': {
                            'background-color': '#80afd6',
                            'line-color': '#80afd6'
                        }
                    },
                    {
                        'selector': '[weight > 0.998]',
                        'style': {
                            'background-color': '#619bcc',
                            'line-color': '#619bcc'
                        }
                    },
                    {
                        'selector': '[weight > 1.151]',
                        'style': {
                            'background-color': '#4288c2',
                            'line-color': '#4288c2'
                        }
                    },
                    {
                        'selector': '[weight > 1.304]',
                        'style': {
                            'background-color': '#3573a6',
                            'line-color': '#3573a6'
                        }
                    },
                    {
                        'selector': '[weight > 1.457]',
                        'style': {
                            'background-color': '#2b5d87',
                            'line-color': '#2b5d87'
                        }
                    },
                    {
                        'selector': '[weight > 1.61]',
                        'style': {
                            'background-color': '#214868',
                            'line-color': '#214868'
                        }
                    },
                    {
                        'selector': '[weight > 1.762]',
                        'style': {
                            'background-color': '#17334a',
                            'line-color': '#17334a'
                        }
                    },
                    {
                        'selector': '[weight > 1.915]',
                        'style': {
                            'background-color': '#0d1d2b',
                            'line-color': '#0d1d2b'
                        }
                    },
                    {
                    'selector': '.rectangle',
                    'style': {
                        'shape': 'rectangle'}
                    },
                    {
                    'selector': '.purple',
                    'style': {
                        'background-color': '#7B5EC6'}
                    }],
                style={'width': '100%', 'height': '100%'},
                )
        ],
            style={
                'height': '42rem',
            }
        )
    ])

In [20]:
prtiscard1 = dbc.Card(
    [
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.Label("Белок — Ткани",
                               style={'font-size': 24,
                                      'text-align': 'left',
                                      },
                               ),
                ], width=6),
                dbc.Col([
                    prottis_filter1,
                ], width=6)
            ]),
            cyto.Cytoscape(
                id='cytoscape-prtiscard1-callbacks',
                layout={'name': 'cose'},
                stylesheet=[
                    {'selector': 'node',
                     'style': {
                         'background-color': '#00BFFF',
                         'label': 'data(label)'}
                    },
                    {'selector': 'edge',
                     'style': {
                         'line-color': '#A3C4BC'}
                    },
                    {
                    'selector': '.rectangle',
                    'style': {
                        'shape': 'rectangle'}
                    },
                    {
                    'selector': '.triangle',
                    'style': {
                        'shape': 'triangle'}
                    },
                    {
                    'selector': '.purple',
                    'style': {
                        'background-color': '#7B5EC6'}
                    },
                    {
                    'selector': '.pink',
                    'style': {
                        'background-color': '#a537ab'}
                    }],
                style={'width': '100%', 'height': '100%'},
                )
        ],
            style={
                'height': '42rem',
            }
        )
    ])

In [21]:
disprotcard1 = dbc.Card(
    [
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.Label("Заболевание — Белки",
                               style={'font-size': 24,
                                      'text-align': 'left',
                                      },
                               ),
                ], width=6),
                dbc.Col([
                    disprot_filter1,
                ], width=6)
            ]),
            cyto.Cytoscape(
                id='cytoscape-disprcard1-callbacks',
                layout={'name': 'cose'},
                stylesheet=[
                    {'selector': 'node',
                     'style': {
                         'label': 'data(label)'}
                    },
                    {
                        'selector': '[weight > 0]',
                        'style': {
                            'background-color': '#dde9f4',
                            'line-color': '#dde9f4'
                        }
                    },
                    {
                        'selector': '[weight > 0.54]',
                        'style': {
                            'background-color': '#bed6ea',
                            'line-color': '#bed6ea'
                        }
                    },
                    {
                        'selector': '[weight > 0.693]',
                        'style': {
                            'background-color': '#9fc2e0',
                            'line-color': '#9fc2e0'
                        }
                    },
                    {
                        'selector': '[weight > 0.846]',
                        'style': {
                            'background-color': '#80afd6',
                            'line-color': '#80afd6'
                        }
                    },
                    {
                        'selector': '[weight > 0.998]',
                        'style': {
                            'background-color': '#619bcc',
                            'line-color': '#619bcc'
                        }
                    },
                    {
                        'selector': '[weight > 1.151]',
                        'style': {
                            'background-color': '#4288c2',
                            'line-color': '#4288c2'
                        }
                    },
                    {
                        'selector': '[weight > 1.304]',
                        'style': {
                            'background-color': '#3573a6',
                            'line-color': '#3573a6'
                        }
                    },
                    {
                        'selector': '[weight > 1.457]',
                        'style': {
                            'background-color': '#2b5d87',
                            'line-color': '#2b5d87'
                        }
                    },
                    {
                        'selector': '[weight > 1.61]',
                        'style': {
                            'background-color': '#214868',
                            'line-color': '#214868'
                        }
                    },
                    {
                        'selector': '[weight > 1.762]',
                        'style': {
                            'background-color': '#17334a',
                            'line-color': '#17334a'
                        }
                    },
                    {
                        'selector': '[weight > 1.915]',
                        'style': {
                            'background-color': '#0d1d2b',
                            'line-color': '#0d1d2b'
                        }
                    },
                    {
                    'selector': '.rectangle',
                    'style': {
                        'shape': 'rectangle'}
                    },
                    {
                    'selector': '.purple',
                    'style': {
                        'background-color': '#7B5EC6'}
                    }],
                style={'width': '100%', 'height': '100%'},
                )
        ],
            style={
                'height': '42rem',
            }
        )
    ])

## Заполнение страницы

In [22]:
app.layout = html.Div(children=[
    html.H1('Взаимосвязь белков с заболеваниями и тканями'),
    html.Br(),
    dbc.Row([
        dbc.Col([protnametype])
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            prdiscard1,
        ]),
        dbc.Col([
            prtiscard1,
        ]),
        dbc.Col([
            disprotcard1,
        ])])
], style={'margin-left':'80px',
         'margin-right':'80px'}
)

## CallBacks

In [23]:
@app.callback(
    Output('protdis_dropdown1', 'options'),
    Input('protnametype_dropdown', 'value')
)
def update_protdis_dropdown(category):
    return get_available_categories(category, mergetable)

In [24]:
@app.callback(
    Output('protdis_dropdown1', 'value'),
    Input('protdis_dropdown1', 'options'))
def set_cities_value(available_prot):
    return available_prot[0]['value']

In [25]:
@app.callback(
    Output('prottis_dropdown1', 'options'),
    Input('protnametype_dropdown', 'value')
)
def update_prottis_dropdown(category):
    return get_available_categories(category, mergetable)

In [26]:
@app.callback(
    Output('prottis_dropdown1', 'value'),
    Input('prottis_dropdown1', 'options'))
def set_cities_value(available_prot):
    return available_prot[0]['value']

In [27]:
@app.callback(
    Output('cytoscape-prdiscard1-callbacks', 'elements'),
    Input('protdis_dropdown1', 'value')
)
def create_dis_net_upd(prot):
    return create_dis_net(prot, mergetable)

In [28]:
@app.callback(
    Output('cytoscape-prtiscard1-callbacks', 'elements'),
    Input('prottis_dropdown1', 'value')
)
def create_tis_net_upd(prot):
    return create_tis_net(prot, mergetable)

In [29]:
@app.callback(
    Output('cytoscape-disprcard1-callbacks', 'elements'),
    [
        Input('disprot_dropdown1', 'value'),
        Input('protnametype_dropdown', 'value')
    ]
)
def create_prot_net_upd(dis, prottype):
    return create_prot_net(dis, mergetable, prottype)

## Запуск приложения

In [ ]:
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [26/Jul/2022 03:42:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2022 03:42:45] "POST /_dash-update-component HTTP/1.1" 200 -
